## M04 Homework

### Michael Vaden, mtv2eva

In [1]:
import pandas as pd
import numpy as np
from glob import glob
import re
import nltk
import plotly_express as px
from lib.textparser import TextParser



In [2]:
import configparser
config = configparser.ConfigParser()
config.read("../env.ini")
data_home = config['DEFAULT']['data_home']
output_dir = config['DEFAULT']['output_dir']

In [3]:
source_files = f'{data_home}/eliot-set'
data_prefix = 'ELIOT_GEORGE'

In [4]:
OHCO = ['book_id', 'chap_num', 'para_num', 'sent_num', 'token_num']

In [5]:
source_file_list = sorted(glob(f"{source_files}/*.*"))

#source_file_list
book_data = []

for source_file_path in source_file_list:
    book_id = int(source_file_path.split('-')[-1].split('.')[0].replace('pg',''))
    book_title = source_file_path.split('/')[-1].split('-')[0].replace('_', ' ')
    book_data.append((book_id, source_file_path, book_title))

LIB = pd.DataFrame(book_data, columns=['book_id','source_file_path','raw_title'])\
    .set_index('book_id').sort_index()
LIB

,source_file_path,raw_title
book_id,,
145,/Users/michaelvaden/GithubRepos/DS5001-Workpla...,ELIOT GEORGE MIDDLEMARCH
507,/Users/michaelvaden/GithubRepos/DS5001-Workpla...,ELIOT GEORGE ADAM BEDE
6688,/Users/michaelvaden/GithubRepos/DS5001-Workpla...,ELIOT GEORGE THE MILL ON THE FLOSS


In [6]:
clip_pats = [
    r"\*\*\*\s*START OF",
    r"\*\*\*\s*END OF"
]

# All are 'chap'and 'm'
roman = '[IVXLCM]+'
caps = "[A-Z';, -]+"
ohco_pat_list = [
    (145, rf"^\s*CHAPTER\s+{roman}\.s*$"),
    (507,   rf"^\s*Chapter\s+{roman}\s*$"),
    (6688,   rf"^\s*Chapter\s+{roman}\.s*$")
]

In [7]:
LIB['chap_regex'] = LIB.index.map(pd.Series({x[0]:x[1] for x in ohco_pat_list}))

In [8]:
LIB

,source_file_path,raw_title,chap_regex
book_id,,,
145,/Users/michaelvaden/GithubRepos/DS5001-Workpla...,ELIOT GEORGE MIDDLEMARCH,^\s*CHAPTER\s+[IVXLCM]+\.s*$
507,/Users/michaelvaden/GithubRepos/DS5001-Workpla...,ELIOT GEORGE ADAM BEDE,^\s*Chapter\s+[IVXLCM]+\s*$
6688,/Users/michaelvaden/GithubRepos/DS5001-Workpla...,ELIOT GEORGE THE MILL ON THE FLOSS,^\s*Chapter\s+[IVXLCM]+\.s*$


In [9]:
def tokenize_collection(LIB):

    clip_pats = [
        r"\*\*\*\s*START OF",
        r"\*\*\*\s*END OF"
    ]

    books = []
    for book_id in LIB.index:

        # Announce
        print("Tokenizing", book_id, LIB.loc[book_id].raw_title)

        # Define vars
        chap_regex = LIB.loc[book_id].chap_regex
        ohco_pats = [('chap', chap_regex, 'm')]
        src_file_path = LIB.loc[book_id].source_file_path

        # Create object
        text = TextParser(src_file_path, ohco_pats=ohco_pats, clip_pats=clip_pats, use_nltk=True)

        # Define parameters
        text.verbose = True
        text.strip_hyphens = True
        text.strip_whitespace = True

        print(text)

        # Parse
        text.import_source().parse_tokens()

        # Name things
        text.TOKENS['book_id'] = book_id
        text.TOKENS = text.TOKENS.reset_index().set_index(['book_id'] + text.OHCO)

        # Add to list
        books.append(text.TOKENS)
        
    # Combine into a single dataframe
    CORPUS = pd.concat(books).sort_index()

    # Clean up
    del(books)
    del(text)
        
    print("Done")
        
    return CORPUS

In [10]:
CORPUS = tokenize_collection(LIB)

Tokenizing 145 ELIOT GEORGE MIDDLEMARCH
Importing  /Users/michaelvaden/GithubRepos/DS5001-Workplace/data/eliot-set/ELIOT_GEORGE_MIDDLEMARCH-pg145.txt
Clipping text
Parsing OHCO level 0 chap_id by milestone ^\s*CHAPTER\s+[IVXLCM]+\.s*$
line_str chap_str
Index(['chap_str'], dtype='object')
Parsing OHCO level 1 para_num by delimitter \n\n
Parsing OHCO level 2 sent_num by NLTK model


Parsing OHCO level 3 token_num by NLTK model
Tokenizing 507 ELIOT GEORGE ADAM BEDE
Importing  /Users/michaelvaden/GithubRepos/DS5001-Workplace/data/eliot-set/ELIOT_GEORGE_ADAM_BEDE-pg507.txt
Clipping text
Parsing OHCO level 0 chap_id by milestone ^\s*Chapter\s+[IVXLCM]+\s*$
line_str chap_str
Index(['chap_str'], dtype='object')
Parsing OHCO level 1 para_num by delimitter \n\n
Parsing OHCO level 2 sent_num by NLTK model
Parsing OHCO level 3 token_num by NLTK model
Tokenizing 6688 ELIOT GEORGE THE MILL ON THE FLOSS
Importing  /Users/michaelvaden/GithubRepos/DS5001-Workplace/data/eliot-set/ELIOT_GEORGE_THE_MILL_ON_THE_FLOSS-pg6688.txt
Clipping text
Parsing OHCO level 0 chap_id by milestone ^\s*Chapter\s+[IVXLCM]+\.s*$
line_str chap_str
Index(['chap_str'], dtype='object')
Parsing OHCO level 1 para_num by delimitter \n\n
Parsing OHCO level 2 sent_num by NLTK model
Parsing OHCO level 3 token_num by NLTK model
Done


A library LIB with the following metadata (and data) about each book:
The book_id, matching the first level of the index in the CORPUS.
The raw book title will be sufficient, i.e. with title and author combined.
The path of the source file.
The regex used to parse chapter milestones.
The length of the book (number of tokens).
The number of chapters in the book.

In [11]:
LIB['book_len'] = CORPUS.groupby('book_id').term_str.count()

LIB['n_chaps'] = CORPUS.reset_index()[['book_id','chap_id']]\
    .drop_duplicates()\
    .groupby('book_id').chap_id.count()

LIB

,source_file_path,raw_title,chap_regex,book_len,n_chaps
book_id,,,,,
145,/Users/michaelvaden/GithubRepos/DS5001-Workpla...,ELIOT GEORGE MIDDLEMARCH,^\s*CHAPTER\s+[IVXLCM]+\.s*$,317305,86
507,/Users/michaelvaden/GithubRepos/DS5001-Workpla...,ELIOT GEORGE ADAM BEDE,^\s*Chapter\s+[IVXLCM]+\s*$,215404,55
6688,/Users/michaelvaden/GithubRepos/DS5001-Workpla...,ELIOT GEORGE THE MILL ON THE FLOSS,^\s*Chapter\s+[IVXLCM]+\.s*$,207461,58


An aggregate of all the novels' tokens CORPUS with an appropriate OHCO index, with following features:
The token string.
The term string.
The part-of-speech tag inferred by NLTK.

In [12]:
CORPUS

pos_tuple  pos  token_str  \
book_id chap_id para_num sent_num token_num                                    
145     1       0        0        0              (Since, IN)   IN      Since   
                                  1                 (I, PRP)  PRP          I   
                                  2                (can, MD)   MD        can   
                                  3                 (do, VB)   VB         do   
                                  4                 (no, DT)   DT         no   
...                                                      ...  ...        ...   
6688    58      69       0        2              (death, NN)   NN      death   
                                  3              (they, PRP)  PRP       they   
                                  4              (were, VBD)  VBD       were   
                                  5                (not, RB)   RB        not   
                                  6          (divided.”, JJ)   JJ  divided.”   

                                            term_str  
book_id chap_id para_num sent_num token_num           
145     1       0        0        0            since  
                                  1                i  
                                  2              can  
                                  3               do  
                                  4               no  
...                                              ...  
6688    58      69       0        2            death  
                                  3             they  
                                  4             were  
                                  5              not  
                                  6          divided  

[740213 rows x 4 columns]

A vocabulary VOCAB of terms extracted from CORPUS, with the following annotation features derived from either NLTK or by using operations presented in the notebook:
Stopwords.
Porter stems.
Maximum POS; i.e. the most frequently associated POS tag for the term using .idxmax(). Note that ties are handled by the method.
POS ambiguity expressed a number of POS tags associated with a term's tokens.

In [13]:
CORPUS = CORPUS[CORPUS.term_str != '']
CORPUS['pos_group'] = CORPUS.pos.str[:2]


VOCAB = CORPUS.term_str.value_counts().to_frame('n').sort_index()
VOCAB.index.name = 'term_str'
VOCAB['n_chars'] = VOCAB.index.str.len()
VOCAB['p'] = VOCAB.n / VOCAB.n.sum()
VOCAB['i'] = -np.log2(VOCAB.p)

VOCAB['max_pos'] = CORPUS[['term_str','pos']].value_counts().unstack(fill_value=0).idxmax(1)
VOCAB['max_pos_group'] = CORPUS[['term_str','pos_group']].value_counts().unstack(fill_value=0).idxmax(1)

VOCAB['n_pos_group'] = CORPUS[['term_str','pos_group']].value_counts().unstack().count(1)
VOCAB['cat_pos_group'] = CORPUS[['term_str','pos_group']].value_counts().to_frame('n').reset_index()\
    .groupby('term_str').pos_group.apply(lambda x: set(x))
VOCAB['n_pos'] = CORPUS[['term_str','pos']].value_counts().unstack().count(1)
VOCAB['cat_pos'] = CORPUS[['term_str','pos']].value_counts().to_frame('n').reset_index()\
    .groupby('term_str').pos.apply(lambda x: set(x))

from nltk.stem.porter import PorterStemmer
stemmer1 = PorterStemmer()
VOCAB['stem_porter'] = VOCAB.apply(lambda x: stemmer1.stem(x.name), 1)

from nltk.stem.snowball import SnowballStemmer
stemmer2 = SnowballStemmer("english")
VOCAB['stem_snowball'] = VOCAB.apply(lambda x: stemmer2.stem(x.name), 1)

from nltk.stem.lancaster import LancasterStemmer
stemmer3 = LancasterStemmer()
VOCAB['stem_lancaster'] = VOCAB.apply(lambda x: stemmer3.stem(x.name), 1)

VOCAB

/var/folders/s6/ny1859490sj527r3gf8tnfrh0000gn/T/ipykernel_90820/2191212758.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  CORPUS['pos_group'] = CORPUS.pos.str[:2]


,n,n_chars,p,i,max_pos,max_pos_group,n_pos_group,cat_pos_group,n_pos,cat_pos,stem_porter,stem_snowball,stem_lancaster
term_str,,,,,,,,,,,,,
1,1,1,0.000001,19.497458,CD,CD,1,{CD},1,{CD},1,1,1
1790,1,4,0.000001,19.497458,CD,CD,1,{CD},1,{CD},1790,1790,1790
1799,2,4,0.000003,18.497458,CD,CD,1,{CD},1,{CD},1799,1799,1799
1801more,1,8,0.000001,19.497458,CD,CD,1,{CD},1,{CD},1801more,1801more,1801more
1807,1,4,0.000001,19.497458,CD,CD,1,{CD},1,{CD},1807,1807,1807
...,...,...,...,...,...,...,...,...,...,...,...,...,...
œdipus,2,6,0.000003,18.497458,NN,NN,1,{NN},1,{NN},œdipu,œdipus,œdip
μέγεθος,1,7,0.000001,19.497458,NNP,NN,1,{NN},1,{NNP},μέγεθος,μέγεθος,μέγεθος
τι,1,2,0.000001,19.497458,NNP,NN,1,{NN},1,{NNP},τι,τι,τι


In [14]:
out_path = f'{output_dir}/{data_prefix}'
LIB.to_csv(f'{out_path}-LIB.csv')
VOCAB.to_csv(f'{out_path}-VOCAB.csv')
CORPUS.to_csv(f'{out_path}-CORPUS.csv')

### Questions

Once you have these, use the dataframes to answer these questions:

What regular expression did you use to chunk _Middlemarch_ into chapters?

What is the title of the book that has the most tokens? 

How many chapter level chunks are there in this novel?

Among the three stemming algorithms -- Porter, Lancaster, and Snowball --  which is the most aggressive, in terms of the number of words associated with each stem?

Using the most aggressive stemmer from the previous question, what is the stem with the most associated terms?

1.What regular expression did you use to chunk _Middlemarch_ into chapters?

In [15]:
LIB.query("book_id == 145")['chap_regex']

book_id
145    ^\s*CHAPTER\s+[IVXLCM]+\.s*$
Name: chap_regex, dtype: object

2.What is the title of the book that has the most tokens?

In [16]:
LIB.sort_values('book_len', ascending=False).iloc[0,:]['raw_title']

'ELIOT GEORGE MIDDLEMARCH'

3.How many chapter level chunks are there in this novel? (Middlemarch)

In [17]:
LIB.sort_values('book_len', ascending=False).iloc[0,:]['n_chaps']

86

4.Among the three stemming algorithms -- Porter, Lancaster, and Snowball --  which is the most aggressive, in terms of the number of words associated with each stem?

In [18]:
print(f"There are {len(VOCAB.stem_porter.unique())} unique stems for Porter")
print(f"There are {len(VOCAB.stem_lancaster.unique())} unique stems for Lancester")
print(f"There are {len(VOCAB.stem_snowball.unique())} unique stems for Snowball")

There are 17540 unique stems for Porter
There are 14612 unique stems for Lancester
There are 17203 unique stems for Snowball


In [19]:
VOCAB.reset_index().groupby(['stem_porter'])['n'].sum().mean()

42.197833523375145

In [20]:
VOCAB.reset_index().groupby(['stem_snowball'])['n'].sum().mean()

43.024472475730974

In [21]:
VOCAB.reset_index().groupby(['stem_lancaster'])['n'].sum().mean()

50.653572406241445

We can see from the results above that **Lancester** has the fewest unique stems and the highest average associated words per stem, so we consider it to be the most aggressive

5.Using the most aggressive stemmer from the previous question, what is the stem with the most associated terms?

In [22]:
VOCAB['stem_lancaster'].value_counts().head()

cont    34
man     27
com     25
adv     21
pass    19
Name: stem_lancaster, dtype: int64

**cont** is the stem  with the most (unique) associated terms